# Carbohydrates Over Time

This notebook generates a chart of carbohydrate consumption measurements over time from data held in the Health Tracker database and retrieved via the Health Tracker Web Service.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import date, datetime, timedelta

# Reporting date range
start = date(2025, 1, 1)
end = datetime.now() + timedelta(days=1)

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_food_consumption_totals(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
import pandas as pd

# Copy the dataframe for subsequent manipulation
df_plot = df.copy()

# Ensure proper types
df_plot["date"] = pd.to_datetime(df_plot["date"], utc=False, errors="coerce")
df_plot = df_plot.dropna(subset=["date", "carbohydrates"])
df_plot = df_plot.sort_values("date")

# Build a continuous daily index with NaNs to prevent lines bridging gaps in the data
daily_full = (
    df_plot.set_index("date")
           .asfreq("D")["carbohydrates"]
)

# Calculate monthly averages
monthly_avg = daily_full.resample("MS").mean().dropna()

In [ ]:
# Export the data to a spreadsheet
if export_spreadsheet:
    export_to_spreadsheet("carbohydrate_consumption", {
        "Daily": daily_full,
        "Monthly": monthly_avg
    })

In [ ]:
# Define carbohydrate bands given chart Y-axis limits
def get_bands(ymax):
    return [
        ("keto", 0, 50),
        ("low", 50, 130),
        ("moderate", 130, 225),
        ("high", 225, ymax)
    ]

# Define band colours for shading chart backgrounds
band_colors = {
    "keto": "C2",       # greenish
    "low": "C1",        # orange
    "moderate": "C4",   # blue
    "high": "C3"        # red/pink
}

In [ ]:
def add_carb_bands(ax, y_upper=None, alpha=0.10):
    """
    Draw background carb bands using distinct colours.
    Safely clips at 225 g so the 'high' band never overwrites 'moderate'
    when the plot's max is below 225.
    """
    # Determine the top we should shade to
    if y_upper is None:
        # fall back to current axis upper bound if not provided
        y_upper = ax.get_ylim()[1]
    top = float(y_upper)

    # Keto: 0–50
    ax.axhspan(0, 50, alpha=alpha, color=band_colors["keto"], zorder=0)

    # Low: 50–130
    ax.axhspan(50, 130, alpha=alpha, color=band_colors["low"], zorder=0)

    # Moderate: 130–min(225, top)
    if top > 130:
        ax.axhspan(130, min(225, top), alpha=alpha, color=band_colors["moderate"], zorder=0)

    # High: 225–top (only if top exceeds 225)
    if top > 225:
        ax.axhspan(225, top, alpha=alpha, color=band_colors["high"], zorder=0)

    # Boundary lines (optional)
    for y in [50, 130, 225]:
        if y < top:
            ax.axhline(y, linewidth=1, linestyle='--', alpha=0.5, color='black', zorder=1)

In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

# Define the chart legend
legend_elements = [
    Line2D([0], [0], color="C0", lw=2, label="Daily carbs"),
    Patch(facecolor=band_colors["keto"], alpha=0.3, label="0–50 g (Keto)"),
    Patch(facecolor=band_colors["low"], alpha=0.3, label="50–130 g (Low carb)"),
    Patch(facecolor=band_colors["moderate"], alpha=0.3, label="130–225 g (Moderate carb)"),
    Patch(facecolor=band_colors["high"], alpha=0.3, label="225+ g (High carb)")
]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MonthLocator

fig1, ax1 = plt.subplots(figsize=(10, 6))

# Actual line
ax1.plot(daily_full.index, daily_full.values, linewidth=1.8, label="Daily carbs")

# Shaded bands
y_upper = max(np.nanmax(daily_full) * 1.15, 250)
add_carb_bands(ax1, y_upper=y_upper, alpha=0.12)

ax1.set_title("Daily Carbohydrate Consumption (g)")
ax1.set_xlabel("Date")
ax1.set_ylabel("Grams per day")

# monthly ticks
ax1.xaxis.set_major_locator(MonthLocator(interval=1))
ax1.xaxis.set_major_formatter(DateFormatter("%b %Y"))
fig1.autofmt_xdate()

# Get the data limits from the data
xmin = daily_full.dropna().index.min()
xmax = daily_full.dropna().index.max()
ax1.set_xlim(xmin, xmax)

ax1.grid(True, axis="y", alpha=0.3)

# Legend positioned below the chart
ax1.legend(
    handles=legend_elements,
    loc="upper center",
    bbox_to_anchor=(0.5, -0.25),   # shift downward
    ncol=3,                        # split legend into columns
    frameon=False
)

# Export to PNG or PDF, if required
export_chart("carbohydrate_consumption_daily", None, chart_export_format)

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MonthLocator

fig2, ax2 = plt.subplots(figsize=(10, 6))

# Draw bands first so they stay behind the bars
y_upper = max(np.nanmax(daily_full) * 1.15, 250)
add_carb_bands(ax2, y_upper=y_upper, alpha=0.12)

# Bars on top
bars = ax2.bar(
    monthly_avg.index,
    monthly_avg.values,
    width=20,           # ~20 days for a nice width on a datetime axis
    zorder=3            # ensure bars sit above the shaded bands/grid
)

# bottom space for legend (tweak as needed)
fig2.subplots_adjust(bottom=0.28)

ax2.set_title("Monthly Average Daily Carbohydrate Consumption (g)")
ax2.set_xlabel("Month")
ax2.set_ylabel("Average grams per day")

# monthly tick formatting
ax2.xaxis.set_major_locator(MonthLocator(interval=1))
ax2.xaxis.set_major_formatter(DateFormatter("%b %Y"))
fig2.autofmt_xdate()

# ✅ set limits from monthly index (with a little padding so first/last bars are fully visible)
left  = monthly_avg.index.min() - pd.Timedelta(days=12)
right = monthly_avg.index.max() + pd.Timedelta(days=12)
ax2.set_xlim(left, right)

# grid behind bars but above spans (spans have default zorder=0)
ax2.grid(True, which="major", axis="y", alpha=0.3, zorder=2)

# Legend positioned below the chart
ax2.legend(
    handles=legend_elements,
    loc="upper center",
    bbox_to_anchor=(0.5, -0.18),
    ncol=3,
    frameon=False
)

# Export to PNG or PDF, if required
export_chart("carbohydrate_consumption_monthly", None, chart_export_format)

plt.show()